In [16]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
import imodelsx.embeddings

df = pd.read_pickle("../data/data_clean.pkl")
n = df.shape[0]

# set up text for prediction
def get_text_representation(row):
    # return f"""- Title: {row["title"]}
# - Description: {row["description"]}
# - Predictor variables: {str(row["feature_names"])[1:-1]}"""
    return f"""{row["title"]}. {row["description"]}. Keywords: {str(row["info___keywords"])[1:-1]}"""
df['text'] = df.apply(get_text_representation, axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
df[['description', 'info___usage___use_case', 'info___usage___why_use', 'info___usage___notes', 'info___next_steps___advice']]

,description,info___usage___use_case,info___usage___why_use,info___usage___notes,info___next_steps___advice
0,"Classifies unexplained events, improves on the...",,,,"<ul>\n<li dir=""ltr"" aria-level=""1"">\n<p dir=""l..."
1,Predicts chance of successful vaginal birth af...,,,,
2,Predicts adverse outcome in adults presenting ...,,,,
3,Identifies low-risk penicillin allergies.,,,,
4,Predicts likelihood of inpatient mortality or ...,,,,
...,...,...,...,...,...
685,Calculates pack years in patients with any smo...,Patients who report any smoking history.,"<ul>\n<li dir=""ltr"">\n<p dir=""ltr"">Standard cl...",,"<span id=""docs-internal-guid-31b99af7-7fff-482..."
686,Determines what immunizations/vaccinations are...,,,This calculator is meant to provide a general ...,Make sure to discuss vaccination and immunizat...
687,Prognoses prostate cancer based on microscopic...,Staging prostate cancer based on biopsy findings.,The Gleason Score is the standard staging crit...,,"Other scores like the <calculator id=""2046"">UC..."
688,Converts steroid dosages using dosing equivale...,<ul><li>Patients on chronic steroids may need ...,Different glucocorticoids possess different du...,<ul><li>These dose relationships only apply to...,Remember to consider not just potency but also...


In [20]:
df.columns

Index(['id', 'title', 'description_short', 'paper___citation', 'paper___url',
       'paper___year', 'participants___male', 'participants___female',
       'participants___total', 'participants___white', 'participants___black',
       'participants___latino', 'participants___asian', 'description',
       'info___mdcalc_url', 'categorization___type', 'title_short',
       'info___before_use', 'info___instructions', 'info___keywords',
       'categorization___chief_complaint', 'categorization___specialty',
       'categorization___purpose', 'categorization___system',
       'categorization___disease', 'feature_names_raw', 'feature_names',
       'paper___author_names', 'paper___author_affiliations',
       'paper___author_genders', 'paper___author_countries',
       'info___related_calc_ids', 'info___additional_references',
       'info___usage___use_case', 'info___usage___why_use',
       'info___usage___notes', 'info___next_steps___advice',
       'info___next_steps___management',
    

In [21]:
def id_to_idx(id, df):
    return np.where(df.id == id)[0]


sims = np.zeros((n, n))
for r, row in tqdm(df.iterrows()):
    ids = row["info___related_calc_ids"]
    for id in ids:
        c = id_to_idx(id, df)
        sims[r, c] += 1

    # for c, col in df.iterrows():
    #     for key in [
    #         "categorization___chief_complaint",
    #         "categorization___specialty",
    #         "categorization___purpose",
    #         "categorization___system",
    #         "categorization___disease",
    #     ]:
    #         if row[key] == col[key]:
    #             sims[r, c] += 1


# average values across the diagonal
sims = (sims + sims.T) / 2

# set diagonal to 1
# np.fill_diagonal(sims, max(sims))

# plot clustermap
# sns.clustermap(sims)

0it [00:00, ?it/s]

690it [00:00, 3750.52it/s]


In [24]:
checkpoint = "bert-base-uncased"
# checkpoint = 'microsoft/deberta-v2-xxlarge'
embs = imodelsx.embeddings.get_embs(df["text"].tolist(), checkpoint, batch_size=32)

100%|██████████| 22/22 [00:02<00:00, 10.02it/s]


In [25]:
embs = imodelsx.embeddings.get_embs_linear(df["text"].tolist())

In [28]:
# calculate pairwise similarity between all embeddings in embs
s = cosine_similarity(embs)

# ranks for largest values in each row (0 is best rank)
ranks = np.argsort(s, axis=1)[:, ::-1]

# matrix showing gt
sims_sym = (sims >= 1.0).astype(bool)

# select ranks for gt
ranks_for_gt = []
for i in range(sims_sym.shape[0]):
    ranks_for_gt.append(ranks[i][sims_sym[i]])

# look at ranks
all_ranks = np.concatenate(ranks_for_gt) - 1
mean_rank = np.mean(all_ranks).round(2)
print(mean_rank, (ranks.shape[1] - 1) / 2)
# plt.axvline(mean_rank, color="k", linestyle="dashed", linewidth=1)
# plt.hist(all_ranks)

324.01 344.5
